### 1. Convert all xlsx to csv
Background:
- Downloaded all datasets related to Passaic River sampling from https://sharepoint.ourpassaic.org/SitePages/Passaic%20River%20Datasets to "xlsx_to_convert" directory. Included: sediment, water column, biota sampling. We will need to review the datasets to determine if they arre alike enough to analyze using the same comparisons.
- Excluded all datasets related to Newark Bay (OU3) because Newark Bay is outside our geographic scope.
- Excluded all Bathymetry (mapping the bottom of the river) datasets because we aren't running an analysis on the physical features or water flow of the River.
- Added all Microsoft Access datasets to a separate directory ("access_to_convert") because those will need a separate conversion process.


Now attempting to add a function to open the xlsx datasets from "xlsx_to_convert" directory and save as csv to the "rawdata" directory
-LK 4/2/2022 at 8am

In [6]:
import pandas as pd
from pathlib import Path
import numpy as np
import os

In [7]:
# Original function to convert excel to csv and save as a new csv file

def convert_xlsx_to_csv(filename):
    filepath = Path(f'../rawdata/xlsx_to_convert/{filename}')
    csv_data = pd.read_excel(filepath)
    new_filename = filepath.stem
    return csv_data.to_csv(f'../rawdata/{new_filename}.csv')

In [8]:
# Testing - it works!
# convert_xlsx_to_csv("1999 Late Summer-Early Fall ESP Sampling.xlsx")

In [9]:
# Now I want to see if we can loop over this folder and convert all of them at once.
# First I'm going to print all the xls files to identify those but also test the loop.
xlsx_directory = '/Users/laurenkrohn/Documents/GitHub-Local/project_1/environmental-contamination/data/rawdata/xlsx_to_convert'
for filename in os.listdir(xlsx_directory):
    if filename.endswith('.xls'):
        print(filename)

In [10]:
# I'm re-writing the function to include the for loop inside
def convert_xlsx_to_csv(xlsx_directory):
    for filename in os.listdir(xlsx_directory):
        if filename.endswith('.xlsx'):
            filepath = Path(f'../rawdata/xlsx_to_convert/{filename}')
            csv_data = pd.read_excel(filepath)
            new_filename = filepath.stem
            csv_data.to_csv(f'../rawdata/{new_filename}.csv')

In [11]:
# I converted the xls files first, then the xlsx files
xlsx_directory = '../rawdata/xlsx_to_convert'
# convert_xlsx_to_csv(xlsx_directory)

### 2. Narrow down the columns in our csvs to just the columns we want
Background:
- In class on 4/2/22 we reviewed the original datasets in excel and identified all columns we want to include in our analysis.
- The columns are:
['LOC_NAME',
'SAMPLE_DATE',
'TASK_CODE', 
'ANALYTIC_METHOD', 
'CAS_RN', 
'CHEMICAL_NAME', 
'REPORT_RESULT_VALUE', 
'REPORT_RESULT_UNIT', 
'REPORT_RESULT_LIMIT', 
'DETECT_FLAG', 
'REPORTABLE_RESULT', 
'LONGITUDE', 
'LATITUDE']
Now we want to write a function that slices the datasets into just those columns. Potential issues are:
- Different column names

In [12]:
# Write function to open csvs
def open_raw_csv(data_directory):
    for filename in os.listdir(data_directory):
        if filename.endswith('.csv'):
            filepath = Path(f'../rawdata/{filename}')
            csv_data = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
            clean_data = csv_data[['LOC_NAME', 'SAMPLE_DATE', 'TASK_CODE', 'ANALYTIC_METHOD', 'CAS_RN', 'CHEMICAL_NAME', 'REPORT_RESULT_VALUE', 'REPORT_RESULT_UNIT', 'REPORT_RESULT_LIMIT', 'DETECT_FLAG', 'REPORTABLE_RESULT', 'LONGITUDE', 'LATITUDE']]
            new_filename = filepath.stem
            clean_data.to_csv(f'../cleandata/{new_filename}.csv')

In [13]:
raw_csv_directory = '../../data/rawdata'
clean_data_directory = '/Users/laurenkrohn/Documents/GitHub-Local/project_1/environmental-contamination/data/cleandata'
contaminants_list = ["2,3,7,8-Tetrachlorodibenzo-p-dioxin","Dieldrin","Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)","Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)","Mercury","Lead","Cyanide"]

In [14]:
def chemical_filter(file_path,chemical_list):
    chemical_data = []

    for filename in os.listdir(file_path):
        if filename.endswith(".csv"):
            csv_data = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True)
            for item in chemical_list:
                chemicals_filtered = csv_data[csv_data['CHEMICAL_NAME'] == item]
                chemicals_filtered = chemicals_filtered.iloc[: , 1:]
                chemical_data.append(chemicals_filtered)

    chemicals_df = pd.concat(chemical_data)
    return chemicals_df

In [17]:
chemical_df = pd.read_csv('chemical_filtered_more.csv', parse_dates=True, infer_datetime_format=True)
chemical_df

,LOC_NAME,SAMPLE_DATE,TASK_CODE,ANALYTIC_METHOD,CAS_RN,CHEMICAL_NAME,REPORT_RESULT_VALUE,REPORT_RESULT_UNIT,REPORT_RESULT_LIMIT,DETECT_FLAG,REPORTABLE_RESULT,LONGITUDE,LATITUDE
0,Hackensack River,7/23/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",48.3,pg/g,0.485,Y,Yes,-74.039502,40.879778
1,Hackensack River,7/30/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",229.0,pg/g,0.481,Y,Yes,-74.069464,40.798054
2,Kill Van Kull,8/20/1981,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",55.4,pg/g,0.495,Y,Yes,-74.098822,40.648703
3,Newark Bay,8/21/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",108.0,pg/g,0.481,Y,Yes,-74.139111,40.671200
4,Newark Bay,4/26/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",113.0,pg/sample,0.671,Y,Yes,-74.139111,40.671200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7761,LPR-0403-01,2019-07-09 11:00:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,400.0,mg/kg,0.990,Y,Yes,-74.145593,40.735957
7762,LPR-0430-07,2019-07-10 10:45:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,370.0,mg/kg,1.100,Y,Yes,-74.149512,40.734412
7763,LPR-0752-01R,2019-07-09 16:10:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,11.0,mg/kg,0.730,Y,Yes,-74.155944,40.770182
7764,LPR-0790-03R,2019-07-09 14:15:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,51.0,mg/kg,0.640,Y,Yes,-74.152242,40.774933
